In [ ]:
import os
import numpy as np
import mne
import scipy
tmax = -0.005
for siteind, site in enumerate(['Tuebingen','Aalto']):
    source_site = rf"D:\REFTEP_ALL\Source_analysis\Source_analysis_{site}"
    for subject in os.listdir(source_site):
        sourcepath_subject =os.path.join(source_site,subject)
        epochs_filepath = os.path.join(sourcepath_subject,f'{subject}_final_eeg-epo.fif')
        epochs = mne.read_epochs(epochs_filepath) #get the respective epochs info for reading the sampling freq
        times = epochs.times
        for aparctype in ['aparc',str(['n15', 'p30', 'n45', 'p60', 'handknob'])]:
            source_estimates_path_subject = os.path.join(sourcepath_subject,f'{subject}_stcs_in_fsaverage_{aparctype}_depth0.8')
            n_files = len([file for file in os.listdir(source_estimates_path_subject) if f"{subject}-fsaverage_{aparctype}_epoch" in file]) #number of files of this type in the folder
            data_all = np.array([np.load(os.path.join(source_estimates_path_subject,f"{subject}-fsaverage_{aparctype}_epoch_{k}.npy")) for k in range(n_files)]) #load all data
            if len(times) != data_all.shape[-1]:
                raise ValueError("Mismatch in time number of samples of times from channel space eeg and source estimate times")
            print(times[(times > -1e-02) & (times < 0)])
            indices = np.array([index for index in range(len(times)) if times[index] <= tmax]) #indices to use from source estimates
            times_cropped = np.array([times[index] for index in range(len(times)) if times[index] <= tmax])
            data = data_all[:,:,indices]
            print(data.shape)
            scipy.io.savemat(os.path.join(sourcepath_subject, f"{subject}_{aparctype}_cropped_mne_depth0.8.mat"),{'source_estimate':data,'times':times_cropped})


